# Creating Agents with Clear Box Orchestration

In this notebook you will learn how to create an Amazon Bedrock Agent using clear box orchestration. We will adapt the hr agent created in feature example #1. The architecture looks as following:

<img src="./images/architecture.png" style="width:70%;display:block;margin: 0 auto;">
<br/>

The steps to complete this notebook are:

1. Import the needed libraries
2. Create the Action Group Lambda 
3. Create the Clear Box Orchestration Lambda
4. Create the Amazon Bedrock Agent
5. Test the Agent
6. Clean-up the resources created

## 1. Import the needed libraries

In [1]:
!python3 -m pip install --force-reinstall --no-cache -r requirements.txt 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.13 -m pip install --upgrade pip
error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this erro

In [2]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.34.162


In [3]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')

session = boto3.session.Session()
region = session.region_name
print(region)

bedrock_agent_client = boto3.client(
    'bedrock-agent',    
    region_name=region
)

[2024-11-20 13:05:40,616] p45001 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


us-east-1


In [5]:
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '007220214296')

In [6]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-clearbox9"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "mistral.mistral-small-2402-v1:0"
agent_description = "Agent for providing HR assistance to manage vacation time"
agent_instruction = """You are an HR agent, helping employees understand HR policies and 
manage vacation time"""
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = """Actions for getting the number of available vactions 
days for an employee and confirm new time off"""
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-ag-lambda-role-{suffix}'
ag_lambda_function_name = f'{agent_name}-ag-{suffix}'
orchestration_lambda_function_name = f'{agent_name}-orchestration-{suffix}'

## 2. Create the Action Group Lambda
We will now create a lambda function that interacts with the SQLite file employee_database.db. To do so we will:
1. Create the employee_database.db file which contains the employee database with some generated data.
2. Create the ag_lambda_function.py file which contains the logic for our lambda action group
3. Create the IAM role for our Lambda function
4. Create the lambda function infrastructure with the required permissions


In [7]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

    # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year - 3, -1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days - days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y-%m-%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) - date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [8]:
%%writefile ag_lambda_function.py
import os
import json
import shutil
import sqlite3
from datetime import datetime

def get_available_vacations_days(employee_id):
    
    if employee_id:

        available_vacation_days = 10

        if available_vacation_days:
            print(f"Available vacation days for employed_id {employee_id}: {available_vacation_days}")
            return available_vacation_days
        else:
            return_msg = f"No vacation data found for employed_id {employee_id}"
            print(return_msg)
            return return_msg
    else:
        raise Exception(f"No employeed id provided")

    # Close the database connection
    conn.close()
    
    
def reserve_vacation_time(employee_id, start_date, end_date):
    # Get the current year
    current_year = start_date.year

    # Check if the employee exists
    if employee is None:
        return_msg = f"Employee with ID {employee_id} does not exist."
        print(return_msg)
        return return_msg

    if available_days is None or available_days[0] < vacation_days:
        return_msg = f"Employee with ID {employee_id} does not have enough vacation days available for the requested period."
        print(return_msg)
        return return_msg

    print(f"Vacation saved successfully for employee with ID {employee_id} from {start_date} to {end_date}.")
    return f"Vacation saved successfully for employee with ID {employee_id} from {start_date} to {end_date}."
        

def lambda_handler(event, context):
    
    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])
    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }


    
    if function == 'get_available_vacations_days':
        employee_id = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]

        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        vacation_days = get_available_vacations_days(employee_id)
        responseBody =  {
            'TEXT': {
                "body": f"available vacation days for employed_id {employee_id}: {vacation_days}"
            }
        }
    elif function == 'reserve_vacation_time':
        employee_id = None
        start_date = None
        end_date = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]
            if param["name"] == "start_date":
                start_date = param["value"]
            if param["name"] == "end_date":
                end_date = param["value"]
            
        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        if not start_date:
            raise Exception("Missing mandatory parameter: start_date")
        if not end_date:
            raise Exception("Missing mandatory parameter: end_date")
        
        completion_message = reserve_vacation_time(employee_id, start_date, end_date)
        responseBody =  {
            'TEXT': {
                "body": completion_message
            }
        }  
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }

    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Writing ag_lambda_function.py


In [9]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)



{'ResponseMetadata': {'RequestId': '72be19f0-cdff-4df0-b22e-1c6b27163ca0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 20 Nov 2024 18:05:50 GMT',
   'x-amzn-requestid': '72be19f0-cdff-4df0-b22e-1c6b27163ca0',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [10]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("ag_lambda_function.py")
z.write("employee_database.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
ag_lambda_function = lambda_client.create_function(
    FunctionName=ag_lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='ag_lambda_function.lambda_handler'
)

## 3. Create the Orchestration Lambda

In [11]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_rewoo_mistral.py") # orchestration lambda code
z.close()
zip_content = s.getvalue()

# Create Lambda Function
orchestration_lambda_function = lambda_client.create_function(
    FunctionName=orchestration_lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_rewoo_mistral.lambda_handler'
)

## 4. Create the Agent

In [12]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/*"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [13]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
            {
                "Sid": "",
                "Effect": "Allow",
                "Principal": {
                    "Service": "preprod.bedrock.aws.internal"
                },
                "Action": "sts:AssumeRole"
            },
            {
                "Sid": "",
                "Effect": "Allow",
                "Principal": {
                    "Service": "beta.bedrock.aws.internal"
                },
                "Action": "sts:AssumeRole"
            },
            {
              "Effect": "Allow",
              "Principal": {
                "Service": "bedrock.amazonaws.com"
              },
              "Action": "sts:AssumeRole"
            }
    ]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'f6bae1ca-77d1-42b5-b4bf-88c8a6a2a7ae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 20 Nov 2024 18:06:02 GMT',
   'x-amzn-requestid': 'f6bae1ca-77d1-42b5-b4bf-88c8a6a2a7ae',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [14]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
    customOrchestration={
        "executor": {
            "lambda": orchestration_lambda_function['FunctionArn']
        }
    },
    orchestrationType="CUSTOM_ORCHESTRATION"
)
response

ParamValidationError: Parameter validation failed:
Unknown parameter in input: "customOrchestration", must be one of: agentName, agentResourceRoleArn, clientToken, customerEncryptionKeyArn, description, foundationModel, guardrailConfiguration, idleSessionTTLInSeconds, instruction, memoryConfiguration, promptOverrideConfiguration, tags
Unknown parameter in input: "orchestrationType", must be one of: agentName, agentResourceRoleArn, clientToken, customerEncryptionKeyArn, description, foundationModel, guardrailConfiguration, idleSessionTTLInSeconds, instruction, memoryConfiguration, promptOverrideConfiguration, tags

In [ ]:
agent_id = response['agent']['agentId']
agent_id

## 4. Create the Agent Action Group

In [ ]:
agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "string"
            }
        }
    },
]


In [ ]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': ag_lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)


In [ ]:
agent_action_group_response

In [ ]:
# Create allow invoke permission on lambda
lambda_client.add_permission(
    FunctionName=ag_lambda_function_name,
    StatementId='allow_bedrock2',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)
lambda_client.add_permission(
    FunctionName=orchestration_lambda_function_name,
    StatementId='allow_bedrock2',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

In [ ]:
# Create allow invoke permission on lambda
lambda_client.add_permission(
    FunctionName=ag_lambda_function_name,
    StatementId='allow_bedrock_preprod2',
    Action='lambda:InvokeFunction',
    Principal='preprod.bedrock.aws.internal',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)
lambda_client.add_permission(
    FunctionName=orchestration_lambda_function_name,
    StatementId='allow_bedrock_preprod2',
    Action='lambda:InvokeFunction',
    Principal='preprod.bedrock.aws.internal',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

In [ ]:
# Create allow invoke permission on lambda
lambda_client.add_permission(
    FunctionName=ag_lambda_function_name,
    StatementId='allow_bedrock_beta2',
    Action='lambda:InvokeFunction',
    Principal='beta.bedrock.aws.internal',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)
lambda_client.add_permission(
    FunctionName=orchestration_lambda_function_name,
    StatementId='allow_bedrock_beta2',
    Action='lambda:InvokeFunction',
    Principal='beta.bedrock.aws.internal',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

In [ ]:
%store orchestration_lambda_function
%store agent_id
%store agent_alias_id

## 5. Test the Agent:

In [ ]:
session = boto3.session.Session()
region = session.region_name
bedrock_agent_runtime_client = boto3.client(
    'bedrock-agent-runtime',
    region_name=region
)

In [ ]:
agent_id, agent_alias_id

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False
session_state={
    'sessionAttributes': {
        'lambda': orchestration_lambda_function['FunctionArn']
    }
}
# invoke the agent API
agent_response = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does employee_id 1 have available?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session,
    sessionState=session_state
)
print(agent_response)
event_stream = agent_response['completion']
try:
    for event in event_stream:
        if 'chunk' in event:
            data = event['chunk']['bytes']
            if enable_trace:
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                for key in event['chunk']:
                    if key != 'bytes':
                        logger.info(f"Chunck {key}:\n")
                        logger.info(json.dumps(event['chunk'][key], indent=3))
            agent_answer = data.decode('utf8')
            print(agent_answer)
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            if enable_trace:
                logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)

except Exception as e:
    raise Exception("unexpected event.", e)

## Clean up (optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. update the action group to disable it
2. delete agent action group
4. delete agent
5. delete both lambda functions
6. delete the created IAM roles and policies


In [ ]:
# This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': ag_lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [ ]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=orchestration_lambda_function_name
)

lambda_client.delete_function(
    FunctionName=ag_lambda_function_name
)

In [ ]:
# Delete IAM Roles and policies

for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)
